In [1]:
#hide
%load_ext autoreload
%autoreload 2    
%load_ext line_profiler
import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore")

In [10]:
#hide
import rfpy
from rfpy.main import parse_input
from rfpy.parser import get_files, export_bin_meta, export_bin_level, file2block, META
from nbdev.showdoc import *
from fastcore.test import *
from fastcore.xtras import Path

# RFPY - Extração otimizada dos dados de arquivos `.bin` provenientes do script Logger executados nas estações de Monitoramento CRFS RFeye Node.
> Este módulo tem como objetivo o processamento e extração otimizada de dados dos arquivos `.bin` de monitoramento do espectro. Para talt utilizamos as várias funcionalidades da biblioteca [fastcore](https://fastcore.fast.ai/basics.html), que expande e otimiza as estruturas de dados da linguagem python. 

## Instalação
`pip install rfpy`

## Como utilizar
Abaixo mostramos as funcionalidades principais dos módulos, utilizando-os dentro de algum outro script ou `REPL`

Precisamos necessariamente de um diretório de entrada, contendo um ou mais arquivos `.bin` e um diretório de saída no qual iremos salvar os arquivos processados. 
> Note: Mude os caminhos abaixo para suas pastas locais caso for executar o exemplo.

Ao utilizar o script `process_bin`, as pastas `entrada` e `saída` esses serão repassadas como parâmetros na linha de comando.

In [3]:
VERBOSE = True
entrada = Path(r'D:\OneDrive - ANATEL\Backup_Rfeye_SP\RPO\PMEC2020\Ribeirao_Preto_SP\SLMA')
saida = Path(r'C:\Users\rsilva\Downloads\saida')

## Leitura de Arquivos

No módulo `parser.py`, há funções auxiliares para lidar com os arquivos `.bin`, pastas e para processar tais arquivos em formatos úteis. Nesse caso utilizaremos a função `get_files` que busca de maneira recursiva arquivos de dada extensão, inclusive links simbólicos se existirem
O caráter recursivo e a busca em links, `recurse` e `followlinks` simbólicos pode ser desativados por meio dos parâmetros e opcionalmente pode ser varrido somente o conjunto de pastas indicado em `folders` 

In [4]:
show_doc(get_files)

<h4 id="get_files" class="doc_header"><code>get_files</code><a href="https://github.com/ronaldokun/rfpy/tree/master/rfpy/parser.py#L42" class="source_link" style="float:right">[source]</a></h4>

> <code>get_files</code>(**`path`**, **`extensions`**=*`None`*, **`recurse`**=*`True`*, **`folders`**=*`None`*, **`followlinks`**=*`True`*)

Get all the files in `path` with optional `extensions`, optionally with `recurse`, only in `folders`, if specified.

In [5]:
arquivos = get_files(entrada, extensions=['.bin']) ; arquivos

(#255) [Path('D:/OneDrive - ANATEL/Backup_Rfeye_SP/RPO/PMEC2020/Ribeirao_Preto_SP/SLMA/rfeye002304_SLMA_bimestral_occ15min_200317_232233.bin'),Path('D:/OneDrive - ANATEL/Backup_Rfeye_SP/RPO/PMEC2020/Ribeirao_Preto_SP/SLMA/rfeye002304_SLMA_bimestral_occ15min_200318_225525.bin'),Path('D:/OneDrive - ANATEL/Backup_Rfeye_SP/RPO/PMEC2020/Ribeirao_Preto_SP/SLMA/rfeye002304_SLMA_bimestral_occ15min_200318_225535.bin'),Path('D:/OneDrive - ANATEL/Backup_Rfeye_SP/RPO/PMEC2020/Ribeirao_Preto_SP/SLMA/rfeye002304_SLMA_bimestral_occ15min_200318_225556.bin'),Path('D:/OneDrive - ANATEL/Backup_Rfeye_SP/RPO/PMEC2020/Ribeirao_Preto_SP/SLMA/rfeye002304_SLMA_bimestral_occ15min_200318_225615.bin'),Path('D:/OneDrive - ANATEL/Backup_Rfeye_SP/RPO/PMEC2020/Ribeirao_Preto_SP/SLMA/rfeye002304_SLMA_bimestral_occ15min_200410_183001.bin'),Path('D:/OneDrive - ANATEL/Backup_Rfeye_SP/RPO/PMEC2020/Ribeirao_Preto_SP/SLMA/rfeye002304_SLMA_bimestral_occ15min_200423_162327.bin'),Path('D:/OneDrive - ANATEL/Backup_Rfeye_SP/RPO/

> Important: O Objeto retornado `L` é uma extensão da lista python com funcionalidades adicionais, uma delas como  podemos ver é que a representação da lista impressa mostra o comprimento da lista. Esse objeto pode ser usado de maneira idêntica à uma lista em python e sem substituição desta.

Temos 255 arquivos bin na pasta entrada. Podemos filtrar por pasta também

In [6]:
arquivos_bin = get_files(entrada, extensions=['.bin'], folders='tmp') ; arquivos_bin

(#1) [Path('D:/OneDrive - ANATEL/Backup_Rfeye_SP/RPO/PMEC2020/Ribeirao_Preto_SP/SLMA/tmp/rfeye002304_SLMA_bimestral_PEAK_200829_234902.bin')]

Nesse caso dentro da pasta 'tmp' há somente 1 arquivo `.bin`

## Processamento dos blocos
A função seguinte recebe um arquivo `.bin` e mapeia os blocos contidos nele.

In [13]:
show_doc(file2block)

<h4 id="file2block" class="doc_header"><code>file2block</code><a href="https://github.com/ronaldokun/rfpy/tree/master/rfpy/parser.py#L91" class="source_link" style="float:right">[source]</a></h4>

> <code>file2block</code>(**`file`**:`Union`\[`str`, `Path`\])

Receives a path to a bin file and returns a defaultdict with unique block types as keys and a list of the Class Blocks as values
:param file: A string or pathlib.Path like path to a `.bin`file generated by CFRS - Logger
:return: A Dictionary with block types as keys and a list of the Class Blocks available as values

In [25]:
bin_file = arquivos_bin[0]
block = file2block(bin_file)

In [26]:
block.keys()

dict_keys([1413563424, 21, 41, 24, 40, 63])

A função seguinte extrai os metadados `META` definidos no cabeçalho do arquivo `parser.py` e retorna um DataFrame.

In [27]:
show_doc(export_bin_meta)

<h4 id="export_bin_meta" class="doc_header"><code>export_bin_meta</code><a href="https://github.com/ronaldokun/rfpy/tree/master/rfpy/parser.py#L123" class="source_link" style="float:right">[source]</a></h4>

> <code>export_bin_meta</code>(**`map_block`**:`Mapping`\[`int`, `L`\])

Receives a Mapping with the different `. bin` Blocks and extracts the metadata listed in [`META`](/rfpy/parser#META) in a dataframe format
    

In [28]:
%%time
meta = export_bin_meta(block)
meta.tail(10)

Wall time: 522 ms


,Block_Number,Latitude,Longitude,Altitude,Initial_Time,Sample_Duration,Start_Frequency,Stop_Frequency,Vector_Length,Trace_Type,Antenna_Type,Equipement_ID
6595,6595,-21.228974,-47.759907,623.7,2020-09-03 13:45:01.833424,60428,108,137,14848,peak,1,rfeye002304
6596,6596,-21.228974,-47.759932,621.4,2020-09-03 13:46:00.811335,60514,108,137,14848,peak,1,rfeye002304
6597,6597,-21.228978,-47.759913,623.0,2020-09-03 13:47:01.501244,60358,108,137,14848,peak,1,rfeye002304
6598,6598,-21.228973,-47.759908,625.1,2020-09-03 13:48:00.511332,60397,108,137,14848,peak,1,rfeye002304
6599,6599,-21.228963,-47.759922,619.9,2020-09-03 13:49:01.191148,60476,108,137,14848,peak,1,rfeye002304
6600,6600,-21.228957,-47.759923,620.5,2020-09-03 13:50:01.891165,60218,108,137,14848,peak,1,rfeye002304
6601,6601,-21.228941,-47.759911,621.1,2020-09-03 13:51:00.811174,60443,108,137,14848,peak,1,rfeye002304
6602,6602,-21.228968,-47.759907,619.1,2020-09-03 13:52:01.501244,60415,108,137,14848,peak,1,rfeye002304
6603,6603,-21.228954,-47.759906,620.3,2020-09-03 13:53:00.411253,60307,108,137,14848,peak,1,rfeye002304
6604,6604,-21.228961,-47.759906,617.8,2020-09-03 13:54:01.091231,60272,108,137,14848,peak,1,rfeye002304


A função seguinte extrai as frequências e nível num formato de Tabela Dinâmica com as Colunas como as Frequências, os números de bloco como índice e os Níveis como valores.

In [29]:
show_doc(export_bin_level)

<h4 id="export_bin_level" class="doc_header"><code>export_bin_level</code><a href="https://github.com/ronaldokun/rfpy/tree/master/rfpy/parser.py#L106" class="source_link" style="float:right">[source]</a></h4>

> <code>export_bin_level</code>(**`map_block`**:`Mapping`\[`int`, `L`\], **`pivoted`**:`bool`=*`True`*)

Receives a mapping `map_block` and returns the Matrix with the Levels as values, Frequencies as columns and Block Number as index.
:param pivoted: If False, optionally returns an unpivoted version of the Matrix

In [30]:
%%time
levels = export_bin_level(block) ; levels.head()

Wall time: 7.01 s


,108.000000,108.001953,108.003907,108.005860,108.007813,108.009766,108.011720,108.013673,108.015626,108.017579,...,136.982421,136.984374,136.986327,136.988280,136.990234,136.992187,136.994140,136.996093,136.998047,137.000000
0,-75.0,-75.5,-76.5,-77.0,-77.5,-78.5,-79.0,-80.5,-80.5,-80.5,...,-95.0,-93.5,-95.0,-93.5,-91.5,-92.0,-98.0,-99.0,-97.5,-96.5
1,-66.0,-70.5,-74.0,-75.5,-75.5,-76.0,-77.0,-77.5,-75.0,-75.5,...,-93.5,-95.0,-94.0,-91.0,-92.0,-94.0,-96.0,-96.5,-96.5,-96.5
2,-83.5,-84.5,-89.5,-81.5,-84.5,-87.0,-85.5,-83.0,-84.5,-87.5,...,-93.5,-93.5,-92.5,-92.0,-92.5,-93.5,-97.5,-98.0,-98.5,-96.5
3,-87.5,-87.5,-91.5,-69.5,-85.5,-85.5,-89.0,-90.5,-88.5,-87.0,...,-93.5,-93.5,-94.0,-93.5,-93.0,-93.0,-98.5,-96.5,-94.5,-95.0
4,-80.5,-75.5,-72.5,-71.5,-73.0,-76.5,-73.0,-74.0,-79.5,-87.5,...,-95.0,-95.0,-94.0,-94.0,-93.5,-93.0,-96.5,-96.5,-95.5,-96.0


Caso o parâmetro `pivoted = False` é retornada a versão tabular empilhada. No entanto o processamento é muito mais lento tendo em vista a redundância de dados que é adicionada.

In [31]:
%%time
levels = export_bin_level(block, pivoted=False) ; levels.head()

AttributeError: 'NoneType' object has no attribute 'reset_index'